I want to predict if an NBA team will make the playoffs. I chose this for my first Machine Learning project because I already know which variables increase the chances of making the playoffs. I am also initerested in the power of the variables when creating the models.

In [556]:
import matplotlib.pyplot as plt  # To visualize
from sklearn.model_selection import GridSearchCV
import matplotlib
matplotlib.use("Qt5Agg")
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
from sklearn import tree
import numpy as np
from sklearn import metrics
pd.options.mode.chained_assignment = None  # default='warn'

First, I am going to initialize some lists and variables that I will use for iteration purposes or throughout the program

In [557]:
# Variables I'll need throughout
path = "/Users/matthewmacrides/Python and R/Github/NBA Playoffs - Machine Learning/NBAPlayoffs"

sep = r'/'

seasons = ['18-19', '19-20', '20-21', '21-22', '22-23']

models = ['Decision Tree', 'Random Forest', 'XGBoost']

I decided to make a for-loop in the case that I add more years of data in the future. All I will need to do is save the .csv file and update the seasons list.

In [558]:
def readData():
     # Create dataframes that will store all the years data
     Nba = pd.DataFrame()

     for season in seasons:
          Nba_ = pd.read_csv(path+sep+season+'.csv', sep=',', decimal='.')
          
          # Create a season column at the end of each file
          Nba_['Season'] = season

          # Not needed because we have win percentage
          Nba_ = Nba_.drop(columns=['Wins', 'Losses'])
          
          # Concatenate files together for each category
          Nba = pd.concat([Nba,Nba_], ignore_index=True)

          Nba.dropna(how='any', inplace=True)

          Teams = Nba[['Team', 'Season']].copy()

          
     return Nba, Teams

I was interested in the correlations across variables. To no surprise, Wins, Losses, Win Percentage, PD, and Standardized PD had strong correlations to Playoffs. I am going to include these variables in my first iteration of the modeliing. Then I am going to remove them to capture the change in accuracy. I am assuming my models will have an accuracy score close to 1 during my first iteration since the variables I mentioned are directly embedded into a team's playoff chances.

In [559]:
Nba, Teams = readData()
Nba.corr().style.background_gradient(cmap="Blues")

,Seed,Win Percentage,PD,FG%,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,Age,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,Playoff,QF,SF,C
Seed,1.000000,-0.955266,-0.907568,-0.413429,-0.449803,-0.284912,-0.357051,-0.276135,-0.066169,-0.176470,0.303913,0.082533,-0.530811,0.108540,0.018050,-0.157708,-0.607603,-0.611579,0.283525,-0.108386,-0.093370,-0.859840,-0.655390,-0.424887,-0.220287
Win Percentage,-0.955266,1.000000,0.956801,0.429759,0.492888,0.289296,0.364871,0.308888,0.059602,0.207207,-0.320959,-0.090218,0.585380,-0.087164,-0.002842,0.170718,0.660299,0.661831,-0.308936,0.056612,0.114534,0.815175,0.623065,0.396235,0.217165
PD,-0.907568,0.956801,1.000000,0.406619,0.508083,0.285598,0.385998,0.322269,0.087397,0.254239,-0.326731,-0.089468,0.583993,-0.083533,0.006368,0.212697,0.673996,0.674720,-0.329642,0.090650,0.123735,0.750590,0.594064,0.390216,0.207743
FG%,-0.413429,0.429759,0.406619,1.000000,-0.042933,0.268809,-0.049736,0.438948,-0.108041,0.055326,-0.034495,-0.115943,0.285548,0.026125,0.035980,-0.160623,0.599523,0.598982,-0.030748,-0.084794,0.137555,0.329774,0.228269,0.194728,0.126432
3P%,-0.449803,0.492888,0.508083,-0.042933,1.000000,0.287064,0.027168,0.206368,-0.112933,0.010138,-0.201252,-0.194409,0.443417,-0.146095,-0.088313,0.212545,0.594610,0.608741,-0.130457,-0.080784,0.024683,0.377987,0.198010,0.162629,0.056596
FT%,-0.284912,0.289296,0.285598,0.268809,0.287064,1.000000,-0.293601,0.042070,-0.137763,-0.116608,-0.392231,-0.157336,0.176423,-0.226895,-0.131292,0.035902,0.458146,0.331504,-0.290433,-0.232868,0.248354,0.270594,0.241557,0.216329,-0.053896
TRB,-0.357051,0.364871,0.385998,-0.049736,0.027168,-0.293601,1.000000,0.090979,-0.062930,0.281654,0.066393,0.024222,0.107631,0.335243,0.012540,0.003927,-0.086076,-0.037828,-0.063939,0.399972,-0.104572,0.291913,0.340347,0.188043,0.138522
AST,-0.276135,0.308888,0.322269,0.438948,0.206368,0.042070,0.090979,1.000000,0.180551,0.149476,0.177346,-0.019539,0.125745,0.216385,-0.230352,-0.002870,0.412200,0.484859,0.098277,0.051963,-0.203166,0.114127,0.131937,0.124743,0.099913
STL,-0.066169,0.059602,0.087397,-0.108041,-0.112933,-0.137763,-0.062930,0.180551,1.000000,0.285208,0.079606,0.124383,-0.133923,0.198683,-0.099915,-0.083493,-0.192611,-0.165242,-0.030711,0.170867,-0.146701,-0.055904,0.172505,0.030273,0.186226
BLK,-0.176470,0.207207,0.254239,0.055326,0.010138,-0.116608,0.281654,0.149476,0.285208,1.000000,-0.026182,0.013781,0.094874,0.283915,-0.002282,-0.032342,0.009374,0.034442,-0.174339,0.039806,-0.050280,0.113234,0.166827,0.065364,0.103196


In [560]:
Nba['Season'].unique()

array(['18-19', '19-20', '20-21', '21-22', '22-23'], dtype=object)

Saving a copy of the Team column so we can join with our test data after the analysis. Also creating a copy of the Nba raw data file in the case that I model whether or not a team will make the quarterfinals. Lastly, I remove all string columns and additional columns that aren't important for this analysis


In [561]:
Nba = Nba.drop(columns=['Team', 'QF', 'SF', 'F', 'C'])

Here, I define a function called 'modeling' to split test and training data. I will chose Playoff as my y value. Playoff will be denoted as a '1' if the team made it and a '0' otherwise. Then I iterated through different models and recorded the accuracy for all of them.

In [601]:
y = Nba[["Playoff", "Season"]]
x = Nba.drop('Playoff', axis=1)
Round = "Playoff"

In [606]:
def modeling(models, x, y, Round):
    # Create x and y datasets with the target variable 'Playoff' in mind. 0 denotes a team does not make the playoffs. 1 denotes that a team does make the playoffs.
    for season in seasons:

        # Separating training and testing data
        season_test = season

        x_train = x[x['Season']!=season_test]
        x_test = x[x['Season']==season_test]
        y_train = y[y['Season']!=season_test]
        y_test = y[y['Season']==season_test]


    datasets = [x_train, x_test, y_train, y_test]
    for dataset in datasets:
        dataset.drop(['Season'], axis=1, inplace=True)

    feature_list = list(x_train.columns)
    
    metric = pd.DataFrame()
    compare = pd.DataFrame()
    features = pd.DataFrame()
    
    for model in models:
            if model == 'Decision Tree':
                model_ = DecisionTreeClassifier()
            elif model == 'Random Forest':
                model_ = RandomForestClassifier()
            elif model == 'XGBoost':
                model_ = XGBClassifier()
     
            
            model_.fit(x_train, y_train.values.ravel())
            y_pred = model_.predict(x_test)

            # Store the accuracy in a dataframe. Concatenate for each accuracy score and record the model that received the result
            accuracy = metrics.accuracy_score(y_test, y_pred)
            data = {'Accuracy Score': [accuracy], 'Model': [model]}
            accuracy = pd.DataFrame(data)
            metric = pd.concat([metric,accuracy], ignore_index=True)
        

            # Find probabilities
            probability = model_.predict_proba(x_test)
            probability = probability[:, 1]

            # Comparing real vs. expected playoffs
            if Round == "Playoff": compare['Playoff'] = y_test
            if Round == "Playoff": compare['Pred Playoff ' + model] = y_pred
            if Round == "QF": compare['QF'] = y_test
            if Round == "QF": compare['Pred QF ' + model] = y_pred
            if Round == "SF": compare['SF'] = y_test
            if Round == "SF": compare['Pred SF ' + model] = y_pred
            if Round == "C": compare['C'] = y_test
            if Round == "C": compare['Pred C ' + model] = y_pred
            compare['Probability ' + model] = probability

            # Store the variable importances
            # Get numerical feature importances
            importances = list(model_.feature_importances_)
            # List of tuples with variable and importance
            feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
            # Sort the feature importances by most important first
            feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
            features_ = pd.DataFrame(feature_importances, columns=['Variables', 'Importance'])
            features_['Model'] = model
            features_ = features_.head(5)
            features = pd.concat([features, features_], ignore_index=True)
            
    return metric, compare, features

As we can see, the models are almost 100% accurate when predicting the playoff teams. The only reason they aren't 100% is because there are years that non-playoff teams in the west have a better record than playoff teams in the east and vice versa.

In [615]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,0.966667,Decision Tree
1,0.933333,Random Forest
2,1.000000,XGBoost


In [604]:
features

,Variables,Importance,Model
0,Seed,0.87,Decision Tree
1,PD,0.03,Decision Tree
2,3P%,0.03,Decision Tree
3,STL,0.02,Decision Tree
4,Age,0.02,Decision Tree
5,Seed,0.30,Random Forest
6,Win Percentage,0.23,Random Forest
7,PD,0.15,Random Forest
8,eFG%,0.05,Random Forest
9,3P%,0.04,Random Forest


Thiis is just showing the team, if they made the playoffs, and what the models predicted.

In [605]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Probability Decision Tree,Pred Playoff Random Forest,Probability Random Forest,Pred Playoff XGBoost,Probability XGBoost
120,Milwaukee Bucks,22-23,1.0,1.0,1.0,1.0,0.987,1,0.997904
121,Boston Celtics,22-23,1.0,1.0,1.0,1.0,0.996,1,0.994823
122,Philadelphia 76ers,22-23,1.0,1.0,1.0,1.0,0.970,1,0.978847
123,Denver Nuggets,22-23,1.0,1.0,1.0,1.0,0.936,1,0.984011
124,Cleveland Cavaliers,22-23,1.0,1.0,1.0,1.0,0.960,1,0.967689
125,Memphis Grizzlies,22-23,1.0,1.0,1.0,1.0,0.960,1,0.989666
126,Sacramento Kings,22-23,1.0,1.0,1.0,1.0,0.933,1,0.951846
127,New York Knicks,22-23,1.0,1.0,1.0,1.0,0.957,1,0.982958
128,Phoenix Suns,22-23,1.0,1.0,1.0,1.0,0.981,1,0.988910
129,Brooklyn Nets,22-23,1.0,1.0,1.0,1.0,0.917,1,0.976883


In [616]:
NbaSmall, Teams = readData()
NbaSmall = NbaSmall.drop(columns=['Win Percentage', 'PD', 'Seed', 'Team', 'QF', 'SF', 'F', 'C'])
y = NbaSmall[["Playoff", "Season"]]
x = NbaSmall.drop('Playoff', axis=1)

In [618]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,0.600000,Decision Tree
1,0.800000,Random Forest
2,0.733333,XGBoost


In [572]:
features

,Variables,Importance,Model
0,TS%,0.37,Decision Tree
1,FT/FGA,0.15,Decision Tree
2,TOV,0.12,Decision Tree
3,ORB%,0.11,Decision Tree
4,Pace,0.08,Decision Tree
5,3P%,0.16,Random Forest
6,TRB,0.09,Random Forest
7,TS%,0.09,Random Forest
8,Age,0.08,Random Forest
9,eFG%,0.07,Random Forest


In [573]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Probability Decision Tree,Pred Playoff Random Forest,Probability Random Forest,Pred Playoff XGBoost,Probability XGBoost
120,Milwaukee Bucks,22-23,1.0,1.0,1.0,1.0,0.85,1,0.914827
121,Boston Celtics,22-23,1.0,1.0,1.0,1.0,0.96,1,0.998825
122,Philadelphia 76ers,22-23,1.0,1.0,1.0,1.0,0.81,1,0.960897
123,Denver Nuggets,22-23,1.0,1.0,1.0,1.0,0.71,1,0.650013
124,Cleveland Cavaliers,22-23,1.0,1.0,1.0,1.0,0.69,0,0.272644
125,Memphis Grizzlies,22-23,1.0,0.0,0.0,1.0,0.54,1,0.846259
126,Sacramento Kings,22-23,1.0,0.0,0.0,1.0,0.66,0,0.024658
127,New York Knicks,22-23,1.0,0.0,0.0,1.0,0.82,1,0.989030
128,Phoenix Suns,22-23,1.0,0.0,0.0,1.0,0.89,1,0.995383
129,Brooklyn Nets,22-23,1.0,1.0,1.0,1.0,0.75,1,0.954033


In [36]:
import pydot
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=10, max_depth = 3)
rf_small.fit(x_train, y_train.values.ravel())
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [373]:
Nba, Teams = readData()
NbaQF = Nba[Nba['Playoff'] == 1]
NbaQF = NbaQF.drop(columns=['Team', 'Playoff', 'SF', 'F', 'C'])
y = NbaQF[["QF", "Season"]]
x = NbaQF.drop('QF', axis=1)
Round = 'QF'

In [377]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,0.5000,Decision Tree
1,0.5625,Random Forest
2,0.5625,XGBoost
3,0.5625,Logistic Regression


In [378]:
features

,Variables,Importance,Model
0,Win Percentage,0.78,Decision Tree
1,FT/FGA,0.09,Decision Tree
2,3PAr,0.08,Decision Tree
3,FTr,0.06,Decision Tree
4,Seed,0.00,Decision Tree
5,Seed,0.30,Random Forest
6,Win Percentage,0.23,Random Forest
7,PD,0.13,Random Forest
8,TS%,0.06,Random Forest
9,eFG%,0.04,Random Forest


In [379]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,QF,Pred QF Decision Tree,Probability Decision Tree,Pred QF Random Forest,Probability Random Forest,Pred QF XGBoost,Probability XGBoost,Pred QF Logistic Regression,Probability Logistic Regression
120,Milwaukee Bucks,22-23,0.0,1.0,1.0,1.0,0.90,1.0,0.92,1.0,0.90
121,Boston Celtics,22-23,0.0,1.0,1.0,1.0,0.92,1.0,0.99,1.0,0.95
122,Philadelphia 76ers,22-23,0.0,1.0,1.0,1.0,0.98,1.0,0.96,1.0,0.96
123,Denver Nuggets,22-23,0.0,1.0,1.0,1.0,0.95,1.0,0.94,1.0,0.88
124,Cleveland Cavaliers,22-23,0.0,1.0,1.0,1.0,0.87,1.0,0.87,1.0,0.80
125,Memphis Grizzlies,22-23,0.0,1.0,1.0,1.0,0.89,1.0,0.86,1.0,0.90
126,Sacramento Kings,22-23,0.0,1.0,1.0,1.0,0.73,1.0,0.60,1.0,0.63
127,New York Knicks,22-23,0.0,0.0,0.0,0.0,0.27,0.0,0.27,0.0,0.30
128,Phoenix Suns,22-23,0.0,0.0,0.0,0.0,0.44,0.0,0.37,0.0,0.35
129,Brooklyn Nets,22-23,0.0,0.0,0.0,0.0,0.17,0.0,0.15,0.0,0.19


In [382]:
NbaC, Teams = readData()
NbaC = NbaC.drop(columns=['Team', 'Playoff', 'QF', 'F'])
y = NbaC[["C", "Season"]]
x = NbaC.drop('C', axis=1)
Round = 'C'

In [383]:
metric, compare, features = modeling(models, x, y, Round)
metric

,Accuracy Score,Model
0,1.0,Decision Tree
1,1.0,Random Forest
2,1.0,XGBoost
3,1.0,Logistic Regression


In [384]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,C,Pred C Decision Tree,Probability Decision Tree,Pred C Random Forest,Probability Random Forest,Pred C XGBoost,Probability XGBoost,Pred C Logistic Regression,Probability Logistic Regression
120,Milwaukee Bucks,22-23,0.0,0.0,0.0,0.0,0.17,0.0,0.14,0.0,0.10
121,Boston Celtics,22-23,0.0,0.0,0.0,0.0,0.13,0.0,0.15,0.0,0.16
122,Philadelphia 76ers,22-23,0.0,0.0,0.0,0.0,0.08,0.0,0.05,0.0,0.07
123,Denver Nuggets,22-23,0.0,0.0,0.0,0.0,0.11,0.0,0.14,0.0,0.10
124,Cleveland Cavaliers,22-23,0.0,0.0,0.0,0.0,0.09,0.0,0.09,0.0,0.10
125,Memphis Grizzlies,22-23,0.0,0.0,0.0,0.0,0.04,0.0,0.03,0.0,0.02
126,Sacramento Kings,22-23,0.0,0.0,0.0,0.0,0.10,0.0,0.11,0.0,0.07
127,New York Knicks,22-23,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00
128,Phoenix Suns,22-23,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.00
129,Brooklyn Nets,22-23,0.0,0.0,0.0,0.0,0.06,0.0,0.07,0.0,0.09
